# Model debugging

Debug file. Use `models/<versions>.py` to define the actual model architecture

In [1]:
import torch
import torch.nn as nn
from torchvision import models

In [126]:
%run models/v3.py

In [127]:
model = ResnetBasedModel()
_ = model.train(True)

In [128]:
n_samples = 4
image_channels = 3
image_width = 512
image_height = 512

image = torch.Tensor(n_samples, image_channels, image_height, image_width)
y, embeddings, activations = model(image)

print("Embedding: ", embeddings.size())
print("Activation: ", activations.size())
print("Output: ", y.size())

Embedding:  torch.Size([4, 2048])
Activation:  torch.Size([4, 14, 16, 16])
Output:  torch.Size([4, 14])


In [129]:
loss = torch.sum(y)
loss

tensor(35.6496, grad_fn=<SumBackward0>)

In [130]:
loss.backward()

In [131]:
result = model.forward_with_gradcam(image, 1)
result.size()

heatmap:  torch.Size([4, 2048, 16, 16])
AMOUNT OF GRADIENTS:  1


torch.Size([4, 16, 16])

In [132]:
(result > 0).sum()

tensor(940)

In [138]:
activations[:, 0, :, :].size()

torch.Size([4, 16, 16])

In [139]:
activations[:, 0, :, :]

tensor([[[ 1.0049,  0.2576,  0.2416,  ..., -0.7908,  1.0698,  0.6669],
         [ 0.8024,  0.1795,  0.0445,  ..., -0.6921,  0.3712, -0.0438],
         [-0.3224, -0.2605,  0.5835,  ...,  0.0626,  0.9583,  0.7600],
         ...,
         [ 0.1553,  0.4223,  0.2587,  ...,  0.1661,  0.4013,  0.6798],
         [-0.1085,  0.3219, -0.4453,  ..., -0.4077, -0.0961, -0.1096],
         [ 0.3524,  0.5333, -0.4206,  ..., -0.6102, -0.4122, -0.4608]],

        [[ 1.0049,  0.2576,  0.2416,  ..., -0.7908,  1.0698,  0.6669],
         [ 0.8024,  0.1795,  0.0445,  ..., -0.6921,  0.3712, -0.0438],
         [-0.3224, -0.2605,  0.5835,  ...,  0.0626,  0.9583,  0.7600],
         ...,
         [ 0.1553,  0.4223,  0.2587,  ...,  0.1661,  0.4013,  0.6798],
         [-0.1085,  0.3219, -0.4453,  ..., -0.4077, -0.0961, -0.1096],
         [ 0.3524,  0.5333, -0.4206,  ..., -0.6102, -0.4122, -0.4608]],

        [[ 1.0049,  0.2576,  0.2416,  ..., -0.7908,  1.0698,  0.6669],
         [ 0.8024,  0.1795,  0.0445,  ..., -0

In [135]:
result

tensor([[[-4.1127e-04, -3.1338e-04, -2.6492e-04,  ..., -2.5139e-04,
          -2.0279e-04, -1.5986e-04],
         [ 1.1067e-05, -1.3445e-04, -1.2210e-05,  ...,  1.5464e-05,
          -8.6622e-05, -3.9833e-06],
         [ 5.7208e-05,  1.0448e-04,  1.3404e-04,  ...,  2.4679e-04,
           2.0033e-04,  1.2104e-04],
         ...,
         [ 4.8094e-05,  5.1740e-05,  4.3456e-05,  ...,  7.8061e-05,
           1.2996e-04,  4.7409e-05],
         [ 2.4123e-05, -1.0610e-05,  5.0073e-06,  ...,  1.2996e-04,
           1.3683e-04,  8.9548e-05],
         [ 2.7741e-05,  6.9134e-05,  1.9296e-05,  ...,  1.1601e-04,
           1.1424e-04,  1.0339e-04]],

        [[-4.1127e-04, -3.1338e-04, -2.6492e-04,  ..., -2.5139e-04,
          -2.0279e-04, -1.5986e-04],
         [ 1.1067e-05, -1.3445e-04, -1.2210e-05,  ...,  1.5464e-05,
          -8.6622e-05, -3.9833e-06],
         [ 5.7208e-05,  1.0448e-04,  1.3404e-04,  ...,  2.4679e-04,
           2.0033e-04,  1.2104e-04],
         ...,
         [ 4.8094e-05,  5

In [31]:
for name, param in model.prediction.named_parameters():
    print(name, param.size())

0.weight torch.Size([14, 2048])
0.bias torch.Size([14])


In [32]:
linear = getattr(model.prediction, "0")

In [35]:
linear.weight.size()

torch.Size([14, 2048])

In [36]:
linear.weight.grad.size()

torch.Size([14, 2048])

In [42]:
pooling = getattr(model.global_pool, "0")
pooling

MaxPool2d(kernel_size=16, stride=16, padding=0, dilation=1, ceil_mode=False)

In [49]:
pooling.__dict__

{'_backend': <torch.nn.backends.thnn.THNNFunctionBackend at 0x7fee05188240>,
 '_backward_hooks': OrderedDict(),
 '_buffers': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 '_parameters': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 'ceil_mode': False,
 'dilation': 1,
 'kernel_size': 16,
 'padding': 0,
 'return_indices': False,
 'stride': 16,
 'training': True}

In [70]:
model.model_ft.layer4

Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (downsample): Sequential(
      (0): Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2

In [71]:
last_bottleneck = getattr(model.model_ft.layer4, "2")
last_bottleneck

Bottleneck(
  (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
)

In [77]:
last_bottleneck.relu.register_hook

AttributeError: 'ReLU' object has no attribute 'register_hook'

In [69]:
getattr(last_bottleneck.downsample, "1").weight.register_hook()

<bound method Tensor.register_hook of Parameter containing:
tensor([0.2392, 0.2747, 0.2593,  ..., 0.1803, 0.2701, 0.2526])>

In [57]:
last_bottleneck.grad

AttributeError: 'Bottleneck' object has no attribute 'grad'